<a href="https://colab.research.google.com/github/lschimidtc/Kaggle-Data-Analysis/blob/main/Nubank_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Necessary libs
import pandas as pd
import pandas_datareader.data as web
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import yfinance as yf
yf.pdr_override()

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Collecting data from Nubank
df = web.get_data_yahoo( 'NU' )

# Dimension
df.shape

[*********************100%***********************]  1 of 1 completed


(127, 6)

In [4]:
# Checking
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-09,11.25,12.24,10.01,10.33,10.33,90160800
2021-12-10,11.45,11.85,9.80,11.85,11.85,50077300
2021-12-13,11.10,11.29,10.31,10.81,10.81,19658400
2021-12-14,10.55,10.58,9.27,9.92,9.92,22728100
2021-12-15,9.63,10.29,9.42,10.01,10.01,12663900


In [5]:
# Start chart figure

# Graphics Grid
figure = make_subplots( 
    rows=2, # number of lines
    cols=2, # number of columns
    shared_xaxes = 'columns', # assign x-axes
    shared_yaxes = 'rows', # assign y-axes
    column_width = [0.8, 0.2], # length 
    row_heights = [0.8, 0.2], # height
    horizontal_spacing = 0.015, # space between columns
    vertical_spacing = 0.1, # space between lines 
    subplot_titles = ['Quotes', 'Price range', 'Volume', ''], # titles in the charts
)

# Atualização na figura
figure.update_layout( 
    template='plotly_dark', # dark theme
    title='NUBANK stock analysis') # title 

In [6]:
# Generate the first chart

# allocating the data down
df['previousClose'] = df['Close'].shift(1)

# setting the color if it is less than yesterday's date
df['color'] = np.where(df['Close'] > df['previousClose'], 'green', 'red')

# set the fill to transparent if close > open and the previously set color otherwise
df['fill'] = np.where(df['Close'] > df['Open'], 'rgba(255, 0, 0, 0)', df['color'])

# subtitles
showlegend = True

# loop
for Data, Linha in df.iterrows():

    # check plot color
    color = dict(fillcolor=Linha['fill'], line=dict(color=Linha['color']))

    # add the Candlestick graphic to the figure
    figure.add_trace(
        go.Candlestick(
            x = [Data], # datas
            open=[Linha['Open']], # open data
            high=[Linha['High']], # high data
            low=[Linha['Low']], # low data
            close=[Linha['Close']], # close data
            increasing=color, # assigning the color
            decreasing=color, # assigning the color
            showlegend=showlegend, # adding subtitles
            name='Quotes', # Titulo
            legendgroup='Hollow Candlesticks' # subtitle group
        ),
        row=1, col=1 # chart position
    )

    # enabling subtitles
    showlegend = False

# remove rangeslider from the chart so as not to disturb the volume plot
figure.update_layout(xaxis_rangeslider_visible=False)

In [7]:
# Generate the second chart

# generating the % of the Volume against the whole base
df['Percentage'] = df['Volume'] * 100 / df['Volume'].sum()

# Add the second graphic
figure.add_trace(
    go.Bar(
        x=df.index, # dates index
        y=df['Volume'], # volume
        text=df['Percentage'], # volume % 
        marker_line_color=df['color'], # color
        marker_color=df['fill'], # color
        name='Volume', # title
        texttemplate='%{text:.2f}%', # decimal places
        hoverinfo='x+y', # cartesial position
        textfont=dict(color='white') # text color
    ),
    col=1, row=2, # chart position in figure
)

In [8]:
# Generate the third graphic

# generate a copy
price_range = df.copy()

# round off closing value
price_range['Close'] = price_range['Close'].round()

# generating the table with the closing price
price_range = price_range.groupby('Close', as_index=False)['Volume'].sum()

# generating the %
price_range['Percentage'] = price_range['Volume'] * 100 / price_range['Volume'].sum()

# Adding the graph to the figure
figure.add_trace(
    go.Bar(
        y=price_range['Close'], # close
        x=price_range['Volume'], # volume
        text=price_range['Percentage'], # %
        name='Price range', # title
        orientation='h', # horizontal alignment of the bar graph
        marker_color='yellow', # color 
        texttemplate='%{text:.2f}% @ %{y}', # labels
        hoverinfo='x+y' # cartesial position
    ),
    col=2, row=1, # chart position in figure
)

figure.show()